### Обработка текстов на естественном языке

## 1. Лингвистический анализ:

In [1]:
import natasha
import pymorphy2
import pandas as pd

In [2]:
df = pd.read_csv('data/lenta/lenta-subset.csv')
df

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1999/10/04/tv/,Телеканалы станут вещать по единому тарифу,С 1 января 2000 года все телеканалы будут опла...,Экономика,Все,1999/10/04
1,https://lenta.ru/news/1999/10/04/volkswagen/,"Volkswagen выкупает остатки акций ""Шкоды""",Германский автопромышленный концерн Volkswagen...,Экономика,Все,1999/10/04
2,https://lenta.ru/news/1999/10/04/tumen/,Прибыль Тюменнефтегаза возросла в 10 раз,"Нераспределенная прибыль ОАО ""Тюменнефтегаз"", ...",Экономика,Все,1999/10/04
3,https://lenta.ru/news/1999/10/05/sprint/,Крупнейшее в истории слияние компаний происход...,Две крупнейших телекоммуникационных компании С...,Экономика,Все,1999/10/05
4,https://lenta.ru/news/1999/10/05/volga/,ГАЗ получил четверть обещанного кредита,"ОАО ""ГАЗ"" и Нижегородский банк Сбербанка Росси...",Экономика,Все,1999/10/05
...,...,...,...,...,...,...
197733,https://lenta.ru/news/2018/12/15/oleinik/,Российский боец UFC включен в Книгу рекордов Г...,Российский боец смешанного стиля (MMA) Алексей...,Спорт,Бокс и ММА,2018/12/15
197734,https://lenta.ru/news/2018/12/15/frank_myr/,Бывший чемпион UFC не выдержал кровопролития и...,Американский боец смешанного стиля (MMA) Фрэн...,Спорт,Бокс и ММА,2018/12/15
197735,https://lenta.ru/news/2018/12/15/mebel/,Моуринью сравнил футболистов с мебелью,Главный тренер «Манчестер Юнайтед» Жозе Моурин...,Спорт,Футбол,2018/12/15
197736,https://lenta.ru/news/2018/12/15/putinrap/,Путин предостерег от запретов рэп-концертов,"Президент России Владимир Путин, выступая на з...",Культура,Музыка,2018/12/15


Токенизация (графематический анализ)

In [3]:
doc = natasha.Doc(df.iloc[3].text)
segmenter = natasha.Segmenter()
doc.segment(segmenter)

In [4]:
doc

Doc(text='Две крупнейших телекоммуникационных компании США ..., tokens=[...], sents=[...])

In [5]:
doc.sents

[DocSent(stop=83, text='Две крупнейших телекоммуникационных компании США ..., tokens=[...]),
 DocSent(start=84, stop=315, text='По сообщению агентства Reuters корпорация Sprint ..., tokens=[...]),
 DocSent(start=316, stop=450, text='Сумма намечающейся сделки должна составить 115 ми..., tokens=[...]),
 DocSent(start=451, stop=682, text='Ожидается, что "итоговая" корпорация будет контро..., tokens=[...]),
 DocSent(start=683, stop=750, text='Эта сделка продолжает ряд суперслияний в телефонн..., tokens=[...]),
 DocSent(start=751, stop=854, text='Предыдущим являлась покупка руководителем WorldCo..., tokens=[...]),
 DocSent(start=855, stop=952, text='На нью-йоркской бирже акции Sprint незамедлительн..., tokens=[...]),
 DocSent(start=953, stop=1041, text='В структуре акционерного капитала Sprint участвую..., tokens=[...]),
 DocSent(start=1042, stop=1116, text='Пока неизвестно судьба их пакетов акций, каждый и..., tokens=[...])]

In [6]:
doc.tokens[:5]

[DocToken(stop=3, text='Две'),
 DocToken(start=4, stop=14, text='крупнейших'),
 DocToken(start=15, stop=35, text='телекоммуникационных'),
 DocToken(start=36, stop=44, text='компании'),
 DocToken(start=45, stop=48, text='США')]

In [7]:
doc.sents[0].tokens

[DocToken(stop=3, text='Две'),
 DocToken(start=4, stop=14, text='крупнейших'),
 DocToken(start=15, stop=35, text='телекоммуникационных'),
 DocToken(start=36, stop=44, text='компании'),
 DocToken(start=45, stop=48, text='США'),
 DocToken(start=49, stop=57, text='достигли'),
 DocToken(start=58, stop=72, text='договоренности'),
 DocToken(start=73, stop=74, text='о'),
 DocToken(start=75, stop=82, text='слиянии'),
 DocToken(start=82, stop=83, text='.')]

In [8]:
[x.text for x in doc.sents[0].tokens]

['Две',
 'крупнейших',
 'телекоммуникационных',
 'компании',
 'США',
 'достигли',
 'договоренности',
 'о',
 'слиянии',
 '.']

#### Синтаксический разбор

In [9]:
emb = natasha.NewsEmbedding()
syntax_parser = natasha.NewsSyntaxParser(emb)
doc.parse_syntax(syntax_parser)
doc.sents[0].syntax.print()

  ┌────► Две                  nummod:gov
  │ ┌──► крупнейших           amod
  │ │ ┌► телекоммуникационных amod
  └─└─└─ компании             nsubj
  │ └──► США                  nmod
┌─└───┌─ достигли             
│   ┌─└► договоренности       obj
│   │ ┌► о                    case
│   └►└─ слиянии              nmod
└──────► .                    punct


In [10]:
def extract_basis(syntactic_tree):
    # id2token = {x.id: x for x in syntactic_tree.tokens}
    # Найти главное слово - сказуемое
    root = [x for x in syntactic_tree.tokens if x.rel == 'root'][0]
    # Проверить, есть ли у него какие-то модификаторы (aux)
    aux = [x for x in syntactic_tree.tokens if x.head_id == root.id and (x.rel=='aux' or x.rel=='aux:pass')]
    # Найти подлежащее
    subject = [x for x in syntactic_tree.tokens if x.head_id == root.id and (x.rel == 'nsubj' or x.rel=='nsubj:pass')]
    return subject[0].text, aux[0].text + ' ' + root.text if aux else root.text  

for sent in doc.sents:
    print(extract_basis(sent))

('компании', 'достигли')
('корпорация', 'приняла')
('Сумма', 'должна')


IndexError: list index out of range

##### 1.1.  Используя синтаксический анализатор в составе библиотеки natasha, доработать процедуру выделения основ предложений, чтобы она работала и со сложными предложениями (выделяла все основы сложносочиненных и сложноподчиненных предложений). 
- Пример предложения: Две крупнейших телекоммуникационных компании США достигли договоренности о слиянии

In [11]:
def extract_complex(syntactic_tree):
    complex = []
    roots = [x for x in syntactic_tree.tokens if x.rel == 'root']  
    for root in roots:
        subjects = [x for x in syntactic_tree.tokens if x.head_id == root.id and (x.rel == 'nsubj' or x.rel == 'nsubj:pass')]
        if subjects:
            subject = subjects[0]
            aids = [x for x in syntactic_tree.tokens if x.head_id == root.id and (x.rel == 'aux' or x.rel == 'aux:pass')]
            aux_text = ' '.join([aux.text for aux in aids]) if aids else ''
            complex.append((subject.text, aux_text + ' ' + root.text))
            for conj_root in [x for x in syntactic_tree.tokens if x.head_id == root.id and x.rel == 'conj']:
                aids_conj = [x for x in syntactic_tree.tokens if x.head_id == conj_root.id and (x.rel == 'aux' or x.rel == 'aux:pass')]
                aux_text_conj = ' '.join([aux.text for aux in aids_conj]) if aids_conj else ''
                subjects_conj = [x for x in syntactic_tree.tokens if x.head_id == conj_root.id and (x.rel == 'nsubj' or x.rel == 'nsubj:pass')]
                if subjects_conj:
                    subject_conj = subjects_conj[0]
                    complex.append((subject_conj.text, aux_text_conj + ' ' + conj_root.text))
        return complex if complex else None

for sent in doc.sents:
    complex = extract_complex(sent.syntax)
    if complex:
        for b in complex:
            print(b)

('компании', ' достигли')
('корпорация', ' приняла')
('Сумма', ' должна')
('сделка', ' продолжает')
('покупка', ' являлась')
('акции', ' поднялись')
('BellSouth', ' потеряла')
('Deutsche', ' участвуют')
('судьба', ' неизвестно')


##### 1.2. Разработать функцию, которая бы выделяла из текста упоминания персоналий. Сопоставить множества персоналий, наиболее часто упоминаемых в новостях Экономики за 2000 и 2015 годы.
##### 1.3. Разработать функцию, которая бы выделяла множество действий, совершенных заданной персоналией («Х поручил то-то, Х предложил то-то, что-то было предложено Х»).


In [12]:
from natasha import NamesExtractor, MorphVocab
from collections import defaultdict
from datetime import datetime

In [13]:
def extract_persons(text):
    morph_dict = MorphVocab()
    extractor = NamesExtractor(morph_dict)
    matches = extractor(text)
    return [match.fact.first for match in matches]

def extract_actions_persons(text, person_name):
    doc = natasha.Doc(text)
    doc.segment(segmenter)
    doc.parse_syntax(syntax_parser)
    actions = set()
    for sent in doc.sents:
        main_word = next((token.text.lower() for token in sent.tokens if token.rel == 'nsubj'), None)

        if main_word and main_word == person_name.lower():
            root_tokens = [token.text for token in sent.tokens if token.rel == 'root']
            noun_tokens = [token.text for token in sent.tokens if (token.rel == 'obj' or token.rel == 'obj:pass')]

            root = root_tokens[0] if root_tokens else ''
            noun = ' '.join(noun_tokens) if noun_tokens else ''

            action = f"{main_word} {root} {noun}".strip()
            actions.add(action)

    return actions

def persons_and_actions(year, n):
    df['year'] = pd.to_datetime(df['date']).dt.year
    data_year = df[(df['year'] == year) & (df['topic'] == 'Экономика')]
    data_year['found_persons'] = None 

    for i, row in data_year.iterrows():
        persons_list = extract_persons(row['text'])
        if persons_list:
            data_year.at[i, 'found_persons'] = persons_list[0]
    data_person = pd.DataFrame(data_year['found_persons'][data_year['found_persons'].notna()].tolist())
    person = pd.concat([data_person], ignore_index=True).drop_duplicates()
    print("Персонали: ", person)
    if n == 1:
        for _, row in person[:1].iterrows():
             person_name = row[0]
             print(person_name)
             for _, news_row in data_year[:50].iterrows():
                 text = news_row['text']
                 actions = extract_actions_persons(text, person_name)
                 if actions:
                     print("Действие персоналя:")
                     print('\n'.join(actions))

1.2. Персонали в новостях Экономики за 2000 год:

In [14]:
persons_and_actions(2000, 0)

C:\Users\adevv\AppData\Local\Temp\ipykernel_9568\3346425889.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_year['found_persons'] = None


Персонали:                  0
0        Владимир
5            Билл
7             Рем
12          Борис
13        Евгений
16             из
18         Келера
19        Герхард
21          марта
26         семена
28         Леонид
29          света
34        Жераром
35        Дмитрий
36     Владимиром
40          Ливии
43            мая
46        Михаила
47        Алексей
48           дали
51         Михаил
57         алмазы
58              Ю
61         Джордж
63        Дональд
64      Александр
65       Бурхарда
69       Анатолий
71         Виктор
78             Из
80           овет
81        августа
88          Ямало
91     Русатоммет
108           АВВ
113           Али
117         марки
120  Европаламент


1.2. Персонали в новостях Экономики за 2015 год:

In [15]:
persons_and_actions(2015, 0)

C:\Users\adevv\AppData\Local\Temp\ipykernel_9568\3346425889.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_year['found_persons'] = None


Персонали:               0
0           из
1         ФИЭО
2        ОСАГО
5      Дмитрий
6     Владимир
7         доля
10       РусАл
15     Аркадий
16       Петра
17       Дания
21          Из
22   Нурсултан
25     Алексей
27        Петр
28        свет
31       марта
33        Доля
41        веры
46   Владимира
47      Герман
48        Джим
55      Ксения
58       Афины
61       Альфа
66      финанс
72      Барака
76         том
78      Интеко
80       Борис
81     Роструд
87   Александр
89       Белла
90      Михаил
91         май
92    Тинькофф
96       БРИКС
99       Ольга
105       Афин
106     Пуэрто
107   Бинбанка
108        Том
118      Наиль
122  Екатерина
126    Автодор
132    Евгений
133    августа
136    Николас
139      Игорь
140     Нирадж
146     рудами
148      Барак
149      марка
155       Слим
156       Кубе
160     Ингвар
167    Нуриэль
189    Георгий


1.3. Выберем например Владимира и выведем соверешенные им действия:

In [16]:
persons_and_actions(2000, 1)

C:\Users\adevv\AppData\Local\Temp\ipykernel_9568\3346425889.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_year['found_persons'] = None


Персонали:                  0
0        Владимир
5            Билл
7             Рем
12          Борис
13        Евгений
16             из
18         Келера
19        Герхард
21          марта
26         семена
28         Леонид
29          света
34        Жераром
35        Дмитрий
36     Владимиром
40          Ливии
43            мая
46        Михаила
47        Алексей
48           дали
51         Михаил
57         алмазы
58              Ю
61         Джордж
63        Дональд
64      Александр
65       Бурхарда
69       Анатолий
71         Виктор
78             Из
80           овет
81        августа
88          Ямало
91     Русатоммет
108           АВВ
113           Али
117         марки
120  Европаламент
Владимир
Действие персоналя:
владимир подписал постановление
Действие персоналя:
владимир отметил ряд


### 2. Векторная модель документа:

2.1. Провести анализ того, как качество тематической классификации новости без лемматизации зависит от размера обучающего множества (см. кривые обучения в зависимости от размера обучающего множества). Сопоставить с качеством классификации модели с лемматизацией.


In [38]:
import numpy as np
import torch.nn.functional as F
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.pipeline import make_pipeline

from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt

In [13]:
df.groupby(by='topic').count()
economy = df
economy['target'] = (economy['topic'] == 'Экономика').astype(np.int8)
economy = economy[['text', 'target']]
df.dropna(inplace=True)

In [14]:
vectorizer = CountVectorizer(
    max_df=0.7,    # Делать признаками слова, которые содержатся в не более, чем заданной доле документов
    min_df=10      # Делать признаки из слов, которые содержатся, по крайней мере, в заданном количестве документов
)

In [25]:
X_economy = vectorizer.fit_transform(economy.text)
X_economy_train, X_economy_test, y_economy_train, y_economy_test = train_test_split(X_economy, economy.target, test_size=0.2, random_state=42, stratify=economy.target)
lm_economy = LogisticRegression()
lm_economy.fit(X_economy_train, y_economy_train)

C:\Users\adevv\miniconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [26]:
print('Accuracy:', accuracy_score(y_economy_test, lm_economy.predict(X_economy_test)))
print('ROC_AUC:', roc_auc_score(y_economy_test, lm_economy.predict_proba(X_economy_test)[:, 1]))

Accuracy: 0.9942241149853607
ROC_AUC: 0.999347440144986


#### 3. Вложения (эмбеддинги) слов:
3.1. Загрузить модель эмбеддингов слов, обученную на художественной литературе (см.
https://github.com/natasha/navec). Для выбранного набора слов сопоставить схожие слова
(расположенные рядом в пространстве вложения) и рассуждение по аналогии. Есть ли
разница с моделью вложений, обученной на новостных сообщениях?

In [32]:
import heapq

def euclidean_distance(x, y):
    return np.linalg.norm(x - y)

def cosine_similarity(x, y):
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

def find_closest_words(emb, words, k=5, similarity=cosine_similarity):
    """Поиск k ближайших слов для каждого слова из списка."""
    result = {}
    for word in words:
        word_vector = emb[word]
        word_distances = [(-similarity(emb[other_word], word_vector), other_word) for other_word in emb.vocab.words]
        closest_words = heapq.nsmallest(k, word_distances)
        result[word] = [w for (_, w) in closest_words]
    return result

In [33]:
example_words = ['важное', 'событие']

In [35]:
closest_words_dict = find_closest_words(emb, example_words, k=5, similarity=cosine_similarity)
for word, closest_words in closest_words_dict.items():
    print(f"Ближайшие слова для'{word}': {closest_words}")

C:\Users\adevv\AppData\Local\Temp\ipykernel_9568\299431391.py:7: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))


Ближайшие слова для'важное': ['важное', 'ключевое', 'значение', 'важнейшее', 'самое']
Ближайшие слова для'событие': ['событие', 'знаковое', 'событием', 'знаменательное', 'важное']


In [33]:
import torch
from slovnet.model.emb import NavecEmbedding
from navec import Navec

In [37]:
navec = Navec.load(r'C:\Users\adevv\Downloads\navec.tar')

In [38]:
emb_layer = NavecEmbedding(emb)

C:\Users\adevv\miniconda3\Lib\site-packages\slovnet\model\emb.py:46: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:212.)
  torch.from_numpy(navec.pq.indexes),


In [39]:
input = torch.tensor([1, 2, 3])
emb_layer(input).shape

torch.Size([3, 300])

In [40]:
def similar_words(word, navec, topn=5):
    word_vec = navec[word]
    sw = []

    for other_word in navec.vocab.words:
        if other_word != word:
            other_vec = navec[other_word]
            similarity = np.dot(word_vec, other_vec) / (np.linalg.norm(word_vec) * np.linalg.norm(other_vec))
            sw.append((other_word, similarity))

    sw = sorted(sw, key=lambda x: x[1], reverse=True)

    tsw = sw[:topn]
    return tsw


In [41]:
for search_word in example_words:
    sw = similar_words(search_word, navec, topn=5)
    print(f"\nБлижайшие слова для '{search_word}':", end=' ')
    for word, similarity in sw:
        print(f"{word}", end=', ')
    print() 

C:\Users\adevv\AppData\Local\Temp\ipykernel_9568\2106215094.py:8: RuntimeWarning: invalid value encountered in scalar divide
  similarity = np.dot(word_vec, other_vec) / (np.linalg.norm(word_vec) * np.linalg.norm(other_vec))



Ближайшие слова для 'важное': интересное, самое, серьезное, значение, другое, 

Ближайшие слова для 'событие': происшествие, знаменательное, событием, события, случившееся, 


#### 4. Нейросетевая обработка текстов:

4.1. Тематический классификатор новостей на основе LSTM демонстрирует очень
высокое качество классификации буквально с первой эпохи обучения. Но является ли это
«заслугой» LSTM или качественного набора эмбеддингов? Реализуйте простейший
классификатор сообщений, в котором на вход полносвязному слою передается просто
среднее арифметическое эмбеддингов слов. Сопоставьте качество классификации с
моделью на основе LSTM

In [25]:
import torch.nn as nn
import torch 
from natasha import Segmenter

In [16]:
class SmartTokenizer:
    """Класс для выделения токенов.
    
    Использует библиотеку natasha для токенизации и лемматизации,
    оставляет только существительные и глаголы."""
    
    def __init__(self):
        emb = natasha.NewsEmbedding()
        self.segmenter = natasha.Segmenter()
        self.morph_tagger = natasha.NewsMorphTagger(emb)
        self.morph_vocab = natasha.MorphVocab()
    
    def __call__(self, text):
        doc = natasha.Doc(text)
        doc.segment(self.segmenter)
        doc.tag_morph(self.morph_tagger)
        tokens = []
        for token in doc.tokens:
            if token.pos in ['NOUN', 'VERB']:
                token.lemmatize(morph_vocab)
                tokens.append(token.lemma)
        return tokens

improved_vectorizer = CountVectorizer(
    tokenizer=SmartTokenizer(),
    token_pattern=None,
    max_df=0.7,    # Делать признаками слова, которые содержатся в не более, чем заданной доле документов
    min_df=10      # Делать признаки из слов, которые содержатся, по крайней мере, в заданном количестве документов
)

morph_vocab = natasha.MorphVocab()

In [17]:
df_test = economy[:1000].copy()
df_train = economy[1000:].copy().reset_index(drop=True)

In [43]:
emb = natasha.NewsEmbedding()
segmenter = Segmenter()

def text_to_ids(emb, text: str, length: int) -> torch.tensor:
    """Преобразование строки в тензор с номерами токенов."""
    # Пунктуационные токены (их кодировать не будем)
    punct = [',', '.', ';', ':', '-', '...', '!', '?']
    # Проведем токенизацию текста
    d = natasha.Doc(text)
    d.segment(segmenter)
    # Для каждого токена, который найдется в словаре эмбеддингов подставим
    # его номер, для прочих подставим номер <unk>
    tmp = torch.tensor([emb.vocab.get(x.text.lower(), emb.vocab.unk_id)
                       for x in d.tokens
                       if x.text not in punct][:length])
    # Дополним последовательность (спереди) токенами <pad>
    return torch.nn.functional.pad(tmp, (length - len(tmp), 0), "constant", emb.vocab.pad_id)

In [49]:
X = torch.stack([text_to_ids(emb, x, 50) for x in economy.text], 0)
y = torch.unsqueeze(torch.tensor(economy.target, dtype=torch.float32), 1)

In [51]:
X_test = torch.stack([text_to_ids(emb, x, 50) for x in df_test.text], 0)
Y_test = torch.unsqueeze(torch.tensor(df_test.target, dtype=torch.float32), 1)
data_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X, y), batch_size=8)
test_data_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_test, Y_test), batch_size=16, shuffle=False)

In [52]:
class SimpleLSTMClassifier(nn.Module):
    """Простой классификатор текста."""
    
    def __init__(self):
        super().__init__()
        # Специальная обёртка, позволяющая использовать
        # эмбеддинги natasha как слой сети PyTorch
        self.embedding = NavecEmbedding(emb)
        # LSTM-слой.
        # Обратите внимание на batch_first=True !
        # По умолчанию этот параметр равен False и первая размерность
        # интерпретируется как длина последовательности, а не батча - 
        # если это не поменять, то сеть будет учиться на "каше" из данных
        self.lstm = nn.LSTM(300,   # размерность элемента последовательности (эмбеддинга)
                            30,    # выходная размерность
                            batch_first=True)
        self.fc = nn.Linear(30, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        # Используйте размерность 1, чтобы выбрать последний временной шаг
        x = x[:, -1, :]
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

In [53]:
model = SimpleLSTMClassifier()

In [54]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), 1e-3)

for epoch in range(5):
    model.train()
    for X, y in data_loader:
        optimizer.zero_grad()
        pred_logits = model(X)
        loss = criterion(pred_logits, y)
        loss.backward()
        optimizer.step()
    model.eval()
    preds = []
    with torch.no_grad():
        for X, y in test_data_loader:
            pred_logits = model(X)
            preds.append(torch.sigmoid(pred_logits))
    preds = torch.cat(preds)
    print(f'Epoch {epoch}: train loss={loss.detach().item():.4f}. ' \
          f'Test accuracy {accuracy_score(df_test.target, preds.numpy() > 0.5):.4f} ' \
          f'ROC_AUC {roc_auc_score(df_test.target, preds.numpy()):.4f}')

Epoch 0: train loss=0.6931. Test accuracy 0.9000 ROC_AUC 0.9923
Epoch 1: train loss=0.6931. Test accuracy 0.9030 ROC_AUC 0.9989
Epoch 2: train loss=0.6931. Test accuracy 0.9080 ROC_AUC 0.9957
Epoch 3: train loss=0.6931. Test accuracy 0.9060 ROC_AUC 0.9960
Epoch 4: train loss=0.6931. Test accuracy 0.9260 ROC_AUC 0.9980


Высокое качество классификации, является результатом успешного сочетания эффективной архитектуры LSTM и качественных эмбеддингов

4.2. Проведите анализ ошибок – найдите новости, на которых модель ошибается, и предложите разумные объяснения этому.

In [72]:
def error_analysis():
    model.eval()
    ap = []
    with torch.no_grad():
        for X, y in test_data_loader:
            pred_logits = model(X)
            preds = torch.sigmoid(pred_logits)
            ap.append(preds)

    ap = torch.cat(ap)

    df_test['predicted'] = (ap.numpy() > 0.5).astype(int)
    
    errors = df_test[df_test['target'] != df_test['predicted']]
    print (errors)

In [73]:
pd.set_option('display.max_colwidth', None)
error_analysis()

Объяснение ошибок:
Текст может имееть разные смыслы, т.е. неоднозначен ну и из-за ключивых слов, которые могут быть паралельны

4.3. Усовершенствуйте классификатор, чтобы он осуществлял многоклассовую 
классификацию (классы «Культура», «Спорт», «Экономика»).

In [19]:
class_mapping = {"Культура": 0, "Спорт": 1, "Экономика": 2}
df["target"] = df["topic"].map(class_mapping)

df.dropna(inplace=True)

df = df.sample(frac=1.).reset_index(drop=True)
df_test = df[:1000].copy()
df_train = df[1000:].copy().reset_index(drop=True)

In [20]:
X = torch.stack([text_to_ids(emb, x, 50) for x in df.text], 0)
y = torch.tensor(df.target, dtype=torch.long)

In [21]:
X_test = torch.stack([text_to_ids(emb, x, 50) for x in df_test.text], 0)
y_test = torch.unsqueeze(torch.tensor(df_test.target, dtype=torch.long), 1)
data_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X, y), batch_size=8)
test_data_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_test, y_test), batch_size=16, shuffle=False)

In [31]:
class ImprovedLSTMClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.embedding = NavecEmbedding(emb)
        self.lstm = nn.LSTM(300, 30, batch_first=True)
        self.fc = nn.Linear(30, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.fc(x)
        return x

In [44]:
num_classes = 3
model = ImprLSTMClassifier(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 1e-3)

In [45]:
for epoch in range(5):
    model.train()
    for X, y in data_loader:
        optimizer.zero_grad()
        pred_logits = model(X)
        loss = criterion(pred_logits, y.squeeze())
        loss.backward()
        optimizer.step()
    
    model.eval()
    preds = []
    with torch.no_grad():
        for X, y in test_data_loader:
            pred_logits = model(X)
            preds.append(pred_logits)
    preds = torch.cat(preds)
    
    preds_classes = preds.argmax(dim=1)
    
    roc_auc = roc_auc_score(df_test.target, F.softmax(preds, dim=1).numpy(), multi_class='ovr')
    print(f'Epoch {epoch}: train loss={loss.detach().item():.4f}. ' \
        f'Test accuracy {accuracy_score(df_test.target, preds_classes):.4f} ' \
        f'ROC_AUC {roc_auc:.4f}')

Epoch 0: train loss=0.0273. Test accuracy 0.9870 ROC_AUC 0.9992
Epoch 1: train loss=0.0114. Test accuracy 0.9910 ROC_AUC 0.9996
Epoch 2: train loss=0.0197. Test accuracy 0.9940 ROC_AUC 0.9998
Epoch 3: train loss=0.1614. Test accuracy 0.9910 ROC_AUC 0.9999
Epoch 4: train loss=0.2540. Test accuracy 0.9950 ROC_AUC 1.0000


In [48]:
def errors1_analysis():
    model.eval()
    ap = []
    with torch.no_grad():
        for X, y in test_data_loader:
            pred_logits = model(X)
            preds = F.softmax(pred_logits, dim=1)
            ap.append(preds)

    ap = torch.cat(ap)

    df_test['predicted'] = ap.argmax(dim=1).numpy()
    
    errors= df_test[df_test['target'] != df_test['predicted']]
    print (errors)

In [49]:
errors1_analysis()

                                                 url  \
426         https://lenta.ru/news/2013/02/25/circus/   
473     https://lenta.ru/news/2013/03/13/winemaking/   
607   https://lenta.ru/news/2001/05/28/papa_ukraina/   
620        https://lenta.ru/news/2004/01/13/tarasov/   
765  https://lenta.ru/news/2010/04/06/annileibovitz/   

                                                 title  \
426       Цирку Никулина утвердили аренду в один рубль   
473                     Никита Михалков стал виноделом   
607  Папа Римский проведет в Киеве и Львове правосл...   
620  Бизнесмен Артем Тарасов собирает деньги на диа...   
765     От Анни Лейбовиц потребовали миллион через суд   

                                                  text      topic    tags  \
426  Арендную плату в один рубль за квадратный метр...   Культура   Театр   
473  Актер и режиссер Никита Михалков вместе с парт...  Экономика  Деньги   
607  Папский нунций (посол) в Киеве Николай Этерови...   Культура     Все   
620  Б